In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def masked_softmax(X,valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X,dim=1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens,shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)

    X = d2l.sequence_mask(X.reshape(-1,shape[-1]),valid_lens,value=1e-6)
    return nn.functional.softmax(X.reshape(shape),dim=-1)

In [4]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([2, 2, 3, 3])


tensor([[[0.3831, 0.3070, 0.1550, 0.1550],
         [0.3150, 0.3192, 0.1829, 0.1829]],

        [[0.2001, 0.2195, 0.3981, 0.1823],
         [0.3538, 0.2713, 0.2180, 0.1569]]])

加性注意力  一般来说，当查询和键是不同长度的矢量时，可以使用加性注意力作为评分函数。

In [11]:
#@save
class AdditiveAttention(nn.Module):
    """加性注意力"""
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hidden)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [12]:
queries, keys = torch.normal(0, 1, (2, 1, 20)), torch.ones((2, 10, 2))
# values的小批量，两个值矩阵是相同的
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(
    2, 1, 1)
valid_lens = torch.tensor([2, 6])

attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8,
                              dropout=0.1)
attention.eval()
attention(queries, keys, values, valid_lens)

tensor([2, 6])


tensor([[[19.0151, 20.0151, 21.0151, 22.0151]],

        [[17.8137, 18.8137, 19.8137, 20.8137]]], grad_fn=<BmmBackward0>)

In [13]:
# 缩放点注意力机制
 # 使用点积可以得到计算效率更高的评分函数， 但是点积操作要求查询和键具有相同的长度

In [17]:
class DotProduckAttention(nn.Module):
    def __init__(self,dropout,**kwargs):
        super(DotProduckAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
    # qk向量场长度一样 ，可以考虑做内积并且除以根号d
    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self,queries,keys,values,valid_lens=None):
        d = queries.shape[-1]
        #transpose 转一下 才能乘
        scores = torch.bmm(queries,keys.transpose(1,2)) / math.sqrt(d)
        # batchsize 查询 键值
        print(scores.shape)
        self.attention_weights = masked_softmax(scores,valid_lens)
        return torch.bmm(self.dropout(self.attention_weights),values)

In [18]:
queries = torch.normal(0, 1, (2, 1, 2))
attention = DotProduckAttention(dropout=0.5)
attention.eval()
attention(queries, keys, values, valid_lens)

torch.Size([2, 1, 10])
tensor([2, 6])


tensor([[[17.7218, 18.7218, 19.7218, 20.7218]],

        [[13.6602, 14.6602, 15.6602, 16.6602]]])